In [20]:
import kfp
import kfp.dsl as dsl
import kfp.onprem as onprem
import kfp.components as comp

def echo_op(text):
    return dsl.ContainerOp(
        name='echo',
        image='library/bash:4.4.23',
        command=['sh', '-c'],
        arguments=['echo "$0"', text],
    )  

@dsl.pipeline(
    name='FMnistPipeline',    
    description='mnist '
)
def fmnist_pipeline(learning_rate, dropout_rate, epoch, act, layer,  
                    checkpoint_dir, saved_model_dir, pvc_name, tensorboard_log,
                    name, model_version, namespace):
  
    exit_task = echo_op("Done!")
    
    with dsl.ExitHandler(exit_task): 

        kubeflow_pvc = dsl.PipelineVolume(pvc=str('demo-pvc'))
        
        mnist = dsl.ContainerOp(
            name='FMnist',
            image='kitaeyoo777/sample-job:CA8279D2',
            command=['python', '/app/fmnist-save-model-renew.py'],
            arguments=[
                "--learning_rate", '0.001397',
                "--dropout_rate", '0.18',
                "--epoch", '11',
                "--act", 'sigmoid',
                "--layer", '2',
                "--checkpoint_dir", '/result/training_checkpoints',
                "--saved_model_dir", '/result/saved_model/',
                "--model_version", '0001',
                "--tensorboard_log", '/result/log'                
            ],
            pvolumes={"/result": kubeflow_pvc}
        )
        
        result = dsl.ContainerOp(
            name='list_list',
            image='library/bash:4.4.23',
            command=['ls', '-R', '/result'],
            pvolumes={"/result": mnist.pvolume}
        )
        
        kfserving = dsl.ContainerOp(
            name='kfserving',
            image='kitaeyoo777/kfserving:56F6F7C1',
            command=['python', '/app/KFServing-fairing.py'],
            arguments=[
                "--namespace", 'demo',
                "--storage_uri", "pvc://" +  str('demo-pvc') + "/saved_model",
                "--name", 'kfserving-fmnist'
            ]
        )        
                
        result.after(mnist)
        kfserving.after(result)
    
from kfp_tekton.compiler import TektonCompiler
    
if __name__ == '__main__':
    TektonCompiler().compile(fmnist_pipeline, "fmnist_pipeline.yaml")